In [587]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsapi

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

%matplotlib inline

In [595]:
hits = pd.read_csv("player_stats_07_24_2019.csv")

In [596]:
hits['player_got_hit'] = hits['player_got_hit'].apply(float)

In [599]:
data = hits.iloc[:, 3:-1]
data

,gamesPlayed,groundOuts,runs,doubles,triples,homeRuns,strikeOuts,baseOnBalls,intentionalWalks,hits,...,walksPer9Inn_p5G,whip_p5G,winPercentage_p5G,atBats_h2h,avg_h2h,hits_h2h,obp_h2h,ops_h2h,slg_h2h,pitcher_hitter_opposite_hand
0,98.0,92.0,59.0,16.0,4.0,7.0,68.0,44.0,0.0,108.0,...,3.86,1.57,0.33,4.0,0.500,2.0,0.500,1.000,0.500,1.0
1,12.0,2.0,3.0,0.0,0.0,0.0,4.0,3.0,0.0,4.0,...,3.86,1.57,0.33,3.0,0.000,0.0,0.000,0.000,0.000,1.0
2,88.0,68.0,73.0,29.0,2.0,22.0,55.0,40.0,6.0,104.0,...,3.86,1.57,0.33,7.0,0.429,3.0,0.556,1.127,0.571,0.0
3,95.0,59.0,40.0,18.0,0.0,14.0,84.0,43.0,1.0,72.0,...,3.86,1.57,0.33,3.0,0.000,0.0,0.400,0.400,0.000,0.0
4,76.0,51.0,23.0,10.0,0.0,5.0,39.0,12.0,2.0,43.0,...,3.86,1.57,0.33,3.0,0.000,0.0,0.250,0.250,0.000,1.0
5,82.0,70.0,40.0,13.0,0.0,12.0,33.0,18.0,0.0,70.0,...,3.86,1.57,0.33,10.0,0.000,0.0,0.231,0.231,0.000,0.0
6,92.0,78.0,60.0,16.0,4.0,17.0,87.0,60.0,2.0,97.0,...,3.86,1.57,0.33,4.0,0.250,1.0,0.250,0.750,0.500,1.0
7,58.0,50.0,22.0,7.0,0.0,11.0,27.0,13.0,1.0,48.0,...,3.86,1.57,0.33,10.0,0.400,4.0,0.455,1.255,0.800,0.0
8,74.0,39.0,31.0,9.0,0.0,16.0,69.0,12.0,0.0,49.0,...,3.86,1.57,0.33,9.0,0.333,3.0,0.400,0.844,0.444,1.0
9,63.0,67.0,43.0,21.0,4.0,8.0,67.0,22.0,2.0,77.0,...,3.86,1.57,0.33,9.0,0.222,2.0,0.273,0.829,0.556,0.0


In [600]:
labels = hits.iloc[:, -1]
labels

0      1.0
1      0.0
2      1.0
3      0.0
4      0.0
5      0.0
6      0.0
7      1.0
8      1.0
9      1.0
10     0.0
11     0.0
12     1.0
13     0.0
14     0.0
15     1.0
16     0.0
17     0.0
18     0.0
19     1.0
20     1.0
21     0.0
22     0.0
23     1.0
24     1.0
25     0.0
26     1.0
27     1.0
28     0.0
29     0.0
      ... 
354    0.0
355    1.0
356    0.0
357    1.0
358    0.0
359    1.0
360    0.0
361    0.0
362    0.0
363    0.0
364    1.0
365    0.0
366    0.0
367    1.0
368    1.0
369    0.0
370    0.0
371    0.0
372    0.0
373    1.0
374    0.0
375    1.0
376    0.0
377    0.0
378    0.0
379    1.0
380    0.0
381    0.0
382    0.0
383    0.0
Name: player_got_hit, Length: 384, dtype: float64

In [601]:
# data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.2)
data_train, data_val, labels_train, labels_val = train_test_split(data_train, labels_train, test_size=0.2)

In [602]:
np.mean(labels_train == 0)

0.5053763440860215

In [603]:
np.mean(labels_train == 1)

0.4946236559139785

In [643]:
logreg = LogisticRegression(penalty='l1').fit(data_train, labels_train)

In [644]:
np.mean(logreg.predict(data_train) == labels_train)

0.75806451612903225

In [645]:
np.mean(logreg.predict(data_val) == labels_val)

0.74468085106382975

In [648]:
hits.take(data_val.take(np.argsort(logreg.predict_proba(data_val)[:, 1])[::-1][:20]).index)['Name']

238        Tommy Edman
283     Hunter Renfroe
109     Chris Herrmann
242      Brandon Drury
269        Jeff McNeil
143    Scooter Gennett
4        Gerardo Parra
204         Zach Green
44        Ryan McMahon
29      Howie Kendrick
281      Franmil Reyes
9          Trea Turner
202       Stephen Vogt
301      Tyler Flowers
108        Chad Pinder
227      Melky Cabrera
350      Bryan Holaday
228     Starling Marte
212     Kyle Schwarber
355       Jorge Alfaro
Name: Name, dtype: object

In [649]:
hits_test = pd.read_csv("player_stats_07_25_2019.csv")
hits_test.head()

,Name,ID,Team,gamesPlayed,groundOuts,runs,doubles,triples,homeRuns,strikeOuts,...,walksPer9Inn_p5G,whip_p5G,winPercentage_p5G,atBats_h2h,avg_h2h,hits_h2h,obp_h2h,ops_h2h,slg_h2h,pitcher_hitter_opposite_hand
0,Adeiny Hechavarria,588751,121,51.0,39.0,19.0,6.0,0.0,5.0,22.0,...,3.38,1.5,0.0,0.0,0.00,0.0,0.0,0.00,0.00,1.0
1,Amed Rosario,642708,121,99.0,100.0,47.0,20.0,5.0,11.0,85.0,...,3.38,1.5,0.0,4.0,0.25,1.0,0.4,1.15,0.75,1.0
2,Dominic Smith,642086,121,87.0,39.0,34.0,10.0,0.0,10.0,44.0,...,3.38,1.5,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0
3,J.D. Davis,605204,121,86.0,68.0,31.0,11.0,1.0,9.0,52.0,...,3.38,1.5,0.0,1.0,1.00,1.0,1.0,3.00,2.00,1.0
4,Jeff McNeil,643446,121,88.0,81.0,50.0,27.0,1.0,9.0,48.0,...,3.38,1.5,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0


In [653]:
data_test = hits_test.iloc[:, 3:]

In [657]:
np.argsort(logreg.predict_proba(data_test)[:, 1])[::]

array([200, 192, 194, 193, 191, 195, 196, 197, 201, 190,  53, 199,  52,
        19,  20,  55,  16, 198,  49, 211, 185, 161,   5,  30, 147,  65,
        44,   0, 146,  12, 173, 154,  59,  36,  51, 127,  67,  41, 182,
       184,  69,  23,  71, 128,  10,  56, 143,  63,  14, 120, 176, 144,
         2, 141, 181,  28, 183,  11,  81, 188, 165, 100, 102,   7,  17,
       114, 160, 180, 202, 104, 113,  62,  70, 209,  39,  54, 205, 136,
        13, 103, 206, 175, 139, 155,  61,  66,  18, 124, 109, 150,  34,
       166,  22,   4, 169, 123, 171, 172, 187,  21, 167,  68, 203,  38,
        33,  26, 189, 119, 208, 207, 151, 149, 121, 162, 204, 126,  98,
        58, 164, 135,  40, 105,  57,  86, 138,  90, 156, 145,  29, 170,
       177,  45,  42, 131, 118,  74,  78, 186,   1,  96,  99, 107, 210,
       129, 137,  25,  32, 117,  37, 178,  82, 157,  46, 158, 112,  60,
       116,  47, 110, 179,  15,  88,  31, 159,  95, 115, 212, 122,  64,
        48, 125,  73, 148, 108, 133,   8, 152,  97, 213, 106, 16

## Now the fun begins

In [524]:
num_rows = len(data)
num_feats = len(np.transpose(data))
print(num_rows)
print(num_feats)

365
61


### Convert our arrays to torch tensors

In [526]:
data_train_t = torch.from_numpy(np.array(data_train)).type(torch.FloatTensor)
data_val_t = torch.from_numpy(np.array(data_val)).type(torch.FloatTensor)
data_test_t = torch.from_numpy(np.array(data_test)).type(torch.FloatTensor)

labels_train_t = torch.from_numpy(np.array(labels_train)).type(torch.LongTensor)
labels_val_t = torch.from_numpy(np.array(labels_val)).type(torch.LongTensor)
labels_test_t = torch.from_numpy(np.array(labels_test)).type(torch.LongTensor)

### Then begin training

In [544]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.classify = nn.Sequential(
            nn.Linear(num_feats, 64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 2)
        )
        self.fc1 = nn.Linear(num_feats, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 2)
        
    def forward(self, x):
        x = self.classify(x)
        return x
    
    def predict(self, x):
        predictions = F.softmax(self.forward(x), dim=1)
        return predictions
        

In [561]:
model = NeuralNet()

In [562]:
# Hyperparameters
learning_rate = .001
reg_param = .1
epochs = 250

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=reg_param)

In [563]:
losses = []
for i in range(epochs):
    outputs = model.forward(data_train_t)
    loss = criterion(outputs, labels_train_t)
    losses.append(loss.item())
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [564]:
losses

[11.889802932739258,
 7.239766597747803,
 2.6394290924072266,
 3.178044319152832,
 5.012734413146973,
 4.688254356384277,
 3.0849449634552,
 1.4509106874465942,
 2.0286500453948975,
 2.9778501987457275,
 3.2501485347747803,
 2.8768861293792725,
 2.070610761642456,
 1.401431918144226,
 1.6695631742477417,
 2.3014073371887207,
 2.3349294662475586,
 1.802303433418274,
 1.3331044912338257,
 1.47062087059021,
 1.806520700454712,
 1.9091795682907104,
 1.7038618326187134,
 1.3743048906326294,
 1.2598201036453247,
 1.47334885597229,
 1.6149464845657349,
 1.4568452835083008,
 1.2263911962509155,
 1.2270982265472412,
 1.3652397394180298,
 1.4033716917037964,
 1.286185383796692,
 1.1471657752990723,
 1.158969759941101,
 1.2537978887557983,
 1.2240631580352783,
 1.0896977186203003,
 1.0523244142532349,
 1.123779296875,
 1.1194168329238892,
 1.0204795598983765,
 0.9821571111679077,
 1.0333335399627686,
 1.0132298469543457,
 0.9341956377029419,
 0.9322390556335449,
 0.9587174654006958,
 0.9174445867

In [565]:
# Training accuracy

_, predicted = torch.max(outputs, 1)
np.mean(predicted.numpy() == labels_train)

0.71673819742489275

In [566]:
with torch.no_grad():
    val_outputs = model.forward(data_val_t)
    _, predicted_val = torch.max(val_outputs.data, 1)

In [567]:
# Validation accuracy

np.mean(predicted_val.numpy() == labels_val)

0.72881355932203384

In [578]:
highest_probs_indices_val = np.argsort(F.softmax(val_outputs)[:, 1].numpy())[-25:][::-1]

/Users/rohannarain/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  """Entry point for launching an IPython kernel.


In [579]:
hits.take(np.array(data_val.take(highest_probs_indices_val).index))['Name']

182        Yuli Gurriel
11          Trea Turner
97      Xander Bogaerts
37       Starling Marte
3        Anthony Rendon
89        J.D. Martinez
158         Alex Gordon
231         Nelson Cruz
290        Trey Mancini
220       Eddie Rosario
24         Trevor Story
193      Ramon Laureano
331        Elvis Andrus
246        James McCann
152      Josh Donaldson
224     Jonathan Schoop
327          Tom Murphy
112        Bryce Harper
357        Justin Upton
363       Shohei Ohtani
229         Miguel Sano
73         Tyler Naquin
189       Marcus Semien
122        Amed Rosario
156    Ronald Acuna Jr.
Name: Name, dtype: object

In [506]:
np.take(data_val, highest_probs_indices_val, axis=0)[0]

array([  9.50000000e+01,   1.09000000e+02,   5.20000000e+01,
         2.00000000e+01,   3.00000000e+00,   5.00000000e+00,
         3.40000000e+01,   3.40000000e+01,   1.00000000e+00,
         1.01000000e+02,   0.00000000e+00,   2.91000000e-01,
         3.47000000e+02,   3.53000000e-01,   4.09000000e-01,
         7.63000000e-01,   3.00000000e+00,   6.00000000e+00,
         6.67000000e-01,   7.00000000e+00,   1.58800000e+03,
         3.82000000e+02,   1.42000000e+02,   3.40000000e+01,
         1.26000000e+02,   0.00000000e+00,   1.00000000e+00,
         3.11000000e-01,   1.05000000e+00,   2.00000000e-01,
         6.00000000e+00,   2.50000000e-01,   4.83000000e-01,
         2.33000000e-01,   3.08000000e-01,   2.00000000e+01,
         3.57000000e-01,   8.03000000e-01,   4.46000000e-01,
         1.62000000e-01,   3.92000000e+00,   8.20000000e-01,
         5.23000000e+00,   2.23000000e+00,   2.02000000e+01,
         1.59700000e+01,   4.01000000e+00,   0.00000000e+00,
         6.50000000e-01,